In [1]:
import pandas as pd
import os

In [2]:
os.chdir('c:\\Users\\thuhi\\workspace\\fraud_detection')
os.getcwd()

'c:\\Users\\thuhi\\workspace\\fraud_detection'

In [3]:
df_1day = pd.read_pickle(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data\2018-04-01.pkl")
df_1day.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [4]:
df_1day["TX_DURING_WEEKEND"] = df_1day["TX_DATETIME"].map(lambda x: 1 if x.weekday()>=5 else 0)
df_1day[["TX_DURING_WEEKEND"]].head()

,TX_DURING_WEEKEND
0,1
1,1
2,1
3,1
4,1


In [5]:
def get_CUSTOMER_ID_NB_TX_nDAY_WINDOW(df):
    windows_size = [1,7,30]
    for size in windows_size:
        df[f"NB_TX_{size}DAY_WINDOW"] = (
        df.groupby("CUSTOMER_ID")["TX_DATETIME"]
        .rolling(f"{size}D", on="TX_DATETIME")
        .count()
        .reset_index(level=0, drop=True)
        )
    return df 
df_30day = pd.DataFrame()
for file in os.listdir(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data"):
    temp = pd.read_pickle(os.path.join(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data",file))
    df_30day = pd.concat([df_30day,temp])



In [6]:
result = (df_30day
          .sort_values("TX_DATETIME")
          .groupby("CUSTOMER_ID")
          .rolling("1D", on="TX_DATETIME")
          ["TRANSACTION_ID"].count()
          .reset_index())

result

,CUSTOMER_ID,TX_DATETIME,TRANSACTION_ID
0,0,2018-04-01 07:19:05,1.0
1,0,2018-04-01 18:00:16,2.0
2,0,2018-04-01 19:02:02,3.0
3,0,2018-04-02 08:51:06,3.0
4,0,2018-04-02 14:05:38,4.0
...,...,...,...
1754150,4999,2018-09-29 12:12:38,5.0
1754151,4999,2018-09-29 17:06:15,5.0
1754152,4999,2018-09-30 04:32:41,5.0
1754153,4999,2018-09-30 08:07:27,4.0


In [7]:
df_30day["CUSTOMER_NB_TX_1D"] = result["TRANSACTION_ID"]
df_30day[df_30day["CUSTOMER_ID"]==0][["TX_DATETIME","CUSTOMER_ID","CUSTOMER_NB_TX_1D"]].sort_values(by="TX_DATETIME")

,TX_DATETIME,CUSTOMER_ID,CUSTOMER_NB_TX_1D
1758,2018-04-01 07:19:05,0,1.0
8275,2018-04-01 18:00:16,0,2.0
8640,2018-04-01 19:02:02,0,2.0
12169,2018-04-02 08:51:06,0,4.0
15764,2018-04-02 14:05:38,0,2.0
...,...,...,...
1750390,2018-09-30 13:38:41,0,1.0
1750758,2018-09-30 14:10:21,0,6.0
1751039,2018-09-30 14:34:30,0,2.0
1751272,2018-09-30 14:54:59,0,5.0


In [8]:
NB_TX_FRAUD_DELAY_AND_SIZE = df_30day.groupby("TERMINAL_ID").rolling("8d",on="TX_DATETIME")["TX_FRAUD"].sum()
NB_TX_FRAUD_DELAY = df_30day.groupby("TERMINAL_ID").rolling("7d",on="TX_DATETIME")["TX_FRAUD"].sum()
NB_TX_FRAUD = NB_TX_FRAUD_DELAY_AND_SIZE - NB_TX_FRAUD_DELAY

NB_TX_FRAUD

TERMINAL_ID  TX_DATETIME        
0            2018-04-02 01:00:01    0.0
             2018-04-02 09:49:55    0.0
             2018-04-03 12:14:41    0.0
             2018-04-05 16:47:41    0.0
             2018-04-07 06:05:21    0.0
                                   ... 
9999         2018-09-28 15:34:24    0.0
             2018-09-30 05:36:08    0.0
             2018-09-30 09:00:10    0.0
             2018-09-30 17:47:37    0.0
             2018-09-30 20:50:32    0.0
Name: TX_FRAUD, Length: 1754155, dtype: float64

In [9]:
NB_TX_DELAY_AND_SIZE = df_30day.groupby("TERMINAL_ID").rolling("8d",on="TX_DATETIME")["TX_FRAUD"].count()
NB_TX_DELAY = df_30day.groupby("TERMINAL_ID").rolling("7d",on="TX_DATETIME")["TX_FRAUD"].count()
NB_TX = NB_TX_DELAY_AND_SIZE - NB_TX_DELAY
NB_TX_DELAY_AND_SIZE.reset_index()



,TERMINAL_ID,TX_DATETIME,TX_FRAUD
0,0,2018-04-02 01:00:01,1.0
1,0,2018-04-02 09:49:55,2.0
2,0,2018-04-03 12:14:41,3.0
3,0,2018-04-05 16:47:41,4.0
4,0,2018-04-07 06:05:21,5.0
...,...,...,...
1754150,9999,2018-09-28 15:34:24,10.0
1754151,9999,2018-09-30 05:36:08,7.0
1754152,9999,2018-09-30 09:00:10,8.0
1754153,9999,2018-09-30 17:47:37,9.0


In [10]:
TERMINAL_ID_RISK_nDAY_WINDOW = NB_TX_FRAUD/NB_TX
TERMINAL_ID_RISK_nDAY_WINDOW.fillna(0).reset_index()

,TERMINAL_ID,TX_DATETIME,TX_FRAUD
0,0,2018-04-02 01:00:01,0.0
1,0,2018-04-02 09:49:55,0.0
2,0,2018-04-03 12:14:41,0.0
3,0,2018-04-05 16:47:41,0.0
4,0,2018-04-07 06:05:21,0.0
...,...,...,...
1754150,9999,2018-09-28 15:34:24,0.0
1754151,9999,2018-09-30 05:36:08,0.0
1754152,9999,2018-09-30 09:00:10,0.0
1754153,9999,2018-09-30 17:47:37,0.0


In [11]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_DATETIME')
    
    terminal_transactions.index=terminal_transactions.TX_DATETIME
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TRANSACTION_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

In [12]:
df_30day = get_count_risk_rolling_window(df_30day)

In [13]:
df_30day.loc[:,"TERMINAL_ID_RISK_1DAY_WINDOW"].fillna(0).value_counts().sort_index()

TERMINAL_ID_RISK_1DAY_WINDOW
0.000000    70471
0.000173        1
0.000173        1
0.000173        2
0.000173        1
            ...  
0.012155        5
0.012156        2
0.012157        2
0.012158        3
0.012245        4
Name: count, Length: 36226, dtype: int64

In [14]:
df_30day[["TERMINAL_ID_NB_TX_1DAY_WINDOW","TERMINAL_ID","TX_DATETIME"]].reset_index().sort_values(["TERMINAL_ID","TX_DATETIME"])

,TRANSACTION_ID,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID,TX_DATETIME
9579,9579,0.0,0,2018-04-02 01:00:01
12806,12806,0.0,0,2018-04-02 09:49:55
24184,24184,0.0,0,2018-04-03 12:14:41
46284,46284,0.0,0,2018-04-05 16:47:41
58807,58807,0.0,0,2018-04-07 06:05:21
...,...,...,...,...
1732586,1732586,9720.0,9999,2018-09-28 15:34:24
1745583,1745583,9739.0,9999,2018-09-30 05:36:08
1747229,1747229,9689.0,9999,2018-09-30 09:00:10
1752796,1752796,9718.0,9999,2018-09-30 17:47:37


In [56]:
import os
print(os.getcwd())
os.chdir("..")
print(os.getcwd())

c:\Users\thuhi\workspace\fraud_detection\notebook
c:\Users\thuhi\workspace\fraud_detection


In [ ]:
from src.baseline.features_engineering import Preprocessor

preprocessor = Preprocessor()
preprocessor


TypeError: Preprocessor.__init__() missing 1 required positional argument: 'df'

: 